<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/FTDEMO_LLAMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-env -q
!pip install datasets -q
!pip install transformers -q
!pip install evaluate -q
!pip install bitsandbytes -q
!pip install accelerate -q

In [ ]:
from datasets import load_from_disk
import colab_env

#Prepare the Dataset
dataset = load_from_disk("/content/gdrive/MyDrive/datasets/flight_dataset_tpu")
print(dataset)  # Inspect the dataset

In [8]:
dataset

Dataset({
    features: ['input', 'label'],
    num_rows: 1127
})

In [5]:
dataset[100]

{'input': 'Calculate the distance from ORD to DTW. Departure: 2024-07-10, Aircraft: Airbus A330, Weather: Cloudy',
 'label': 'short'}

In [6]:
dataset[0]

{'input': 'Calculate the distance from DFW to ORD. Departure: 2024-03-25, Aircraft: Airbus A321, Weather: Snowy',
 'label': 'medium'}

In [7]:
dataset[200]

{'input': 'Calculate the distance from DFW to JFK. Departure: 2024-12-18, Aircraft: Airbus A321, Weather: Rainy',
 'label': 'long'}

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
import torch
import bitsandbytes as bnb
import evaluate # Import the evaluate library
from peft import LoraConfig, get_peft_model # Import PEFT modules
import warnings
warnings.filterwarnings("ignore")


# Model and Tokenizer
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Quantization Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# PEFT Configuration (LoRA)
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Label Mapping
label_mapping = {
    "short": 0,
    "medium": 1,
    "long": 2
}

# Tokenize and Format
def tokenize_function(examples):
    examples["labels"] = [label_mapping[label] for label in examples["label"]]
    # Specify a fixed max_length for padding
    tokenized_output = tokenizer(
        examples["input"],
        padding="max_length",
        truncation=True,
        max_length=128  # Set a fixed max_length here
    )
    # Instead of assigning labels directly, shift them for causal LM
    tokenized_output['labels'] = tokenized_output['input_ids'].copy()
    # Replace input_ids corresponding to pad_token with -100 in labels
    tokenized_output['labels'] = [
        [-100 if token == tokenizer.pad_token_id else token for token in label_list]
        for label_list in tokenized_output['labels']
    ]
    return tokenized_output

tokenized_datasets = dataset.map(tokenize_function, batched=True)
print('\n')
print(tokenized_datasets) # Inspect after tokenization
print('\n')

# Remove unnecessary columns
tokenized_datasets = tokenized_datasets.remove_columns(["input", "label"])
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Split Dataset
train_testvalid = tokenized_datasets.train_test_split(test_size=0.2, seed=42)
train_dataset = train_testvalid["train"]
testvalid_dataset = train_testvalid["test"]
test_valid = testvalid_dataset.train_test_split(test_size=0.5, seed=42)
eval_dataset = test_valid["test"]
test_dataset = test_valid["train"]

small_train_dataset = train_dataset.shuffle(seed=42).select(range(800))
small_eval_dataset = eval_dataset.shuffle(seed=42).select(range(113))

# Metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Get predictions for the last non-padded token for each sequence
    predictions = logits[:, :-1, :].argmax(-1)  # Exclude last token (usually pad)
    # Remove -100 from labels before computing accuracy
    labels_cleaned = labels[:, 1:]  # Exclude the first token to align
    labels_cleaned = labels_cleaned[labels_cleaned != -100]
    predictions_cleaned = predictions.flatten()[labels[:, 1:].flatten() != -100]

    return metric.compute(predictions=predictions_cleaned, references=labels_cleaned)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    report_to='none',
    logging_steps=10,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

# Train
trainer.train()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.0622


Dataset({
    features: ['input', 'label', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 1127
})




No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss,Accuracy
10,2.647500,2.563680,0.477778
20,2.368800,2.242467,0.498077
30,2.015200,1.871687,0.596154
40,1.678200,1.540268,0.650000
50,1.394200,1.291684,0.728419
